<a href="https://colab.research.google.com/github/asigalov61/MIDI-TXT-MIDI/blob/master/MIDI_TXT_MIDI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MIDI-TXT-MIDI (Version 3.0)
***
## A full-featured MIDI processor and encoder/decoder for NLP based symbolic music generation and AI model creation/training
***
### Based on absolutely amazing repo-code-colab by Soheil: https://github.com/blu-ray/Music-Generation and on fantastic Python library MIDI.py by Peter Billam https://pjb.com.au/

***

# Supported MIDI features:

## All MIDI.py EVENTS (in an "opus" structure)

***

Project Los Angeles

Tegridy Code 2020


# Preparation

In [ ]:
#@title Download MIDI.py and a test Dataset
#MIDI Library
!curl -L "https://pjb.com.au/midi/free/MIDI.py" > 'MIDI.py'

## MIDI format Mozart Data
!wget http://www.piano-midi.de/zip/mozart.zip
!sudo apt-get install unzip
!unzip mozart.zip -d Dataset

# MIDI to TXT Encoder

***

Sampling = Encode only so many events from each MIDI file

Advanced Events = control_change, pitch_wheel, etc.

Allow Tempo Changes =  Enables/Disables 'set_tempo' encoding

Allow Control Change = Enables/Disables 'control_change' encoding

Karaoke =  Encode text fields (very useful for hybrid models)

***

In [ ]:
#@title Process MIDI to TXT
enable_sampling = False #@param {type:"boolean"}
sample_length_in_MIDI_events = 997 #@param {type:"slider", min:0, max:10000, step:1}
advanced_events = False #@param {type:"boolean"}
allow_tempo_changes = False #@param {type:"boolean"}
allow_control_change = False #@param {type:"boolean"}
karaoke = True #@param {type:"boolean"}




# MIDI Dataset to txt converter 
import MIDI
import os
import numpy as np
import tqdm.auto

def write_notes(file_address):
    midi_file = open(file_address, 'rb')
    #print('Processing File:', file_address)
    score = MIDI.midi2opus(midi_file.read())
    midi_file.close()
    # ['note', start_time, duration, channel, note, velocity]

    itrack = 1


    notes = []

    tokens = []

    this_channel_has_note = False

    file = open('Dataset.txt', 'a')
    file.write('[MIDI-TXT-MIDI Textual Music Dataset] ')
    while itrack < len(score):
        for event in score[itrack]:

            if event[0] == 'note_off':
                this_channel_has_note = True
                notes.append(event[3])

                
                tokens.append([event[3], event[4], event[1]])
                
                file.write('F' + ' d' + str(event[1]) + ' c' + str(event[2]) + ' n' + str(event[3]) + ' v' + str(event[4]) + ' ')


            if event[0] == 'note_on':
                this_channel_has_note = True
                notes.append(event[3])
                
                tokens.append([event[3], event[4], event[1]])

                file.write('N' + ' d' + str(event[1]) + ' c' + str(event[2]) + ' n' + str(event[3]) + ' v' + str(event[4]) + ' ')

            if event[0] == 'key_after_touch':
              if advanced_events:
                this_channel_has_note = True

                
                tokens.append([event[3], event[4], event[1]])
                file.write('K' + ' d' + str(event[1]) + ' c' + str(event[2]) + ' n' + str(event[3]) + ' v' + str(event[4]) + ' ')

            if event[0] == 'control_change':
              if advanced_events:
                  if allow_control_change:
                    this_channel_has_note = True

                
                    tokens.append([event[3], event[4], event[1]])
                
                    file.write('C' + ' d' + str(event[1]) + ' c' + str(event[2]) + ' r' + str(event[3]) + ' l' + str(event[4]) + ' ')

            if event[0] == 'patch_change':
              if advanced_events:
                  this_channel_has_note = True
               
                  tokens.append([event[3], event[2], event[1]])
                
                  file.write('P' + ' d' + str(event[1]) + ' c' + str(event[2]) + ' h' + str(event[3]) + ' ')

            if event[0] == 'channel_after_touch':
              if advanced_events:
                  this_channel_has_note = True

                
                  tokens.append([event[3], event[2], event[1]])
                
                  file.write('Z' + ' d' + str(event[1]) + ' c' + str(event[2]) + ' v' + str(event[3]) + ' ')

            if event[0] == 'pitch_wheel_change':
              if advanced_events:
                this_channel_has_note = True

                
                tokens.append([event[3], event[2], event[1]])
                
                file.write('W' + ' d' + str(event[1]) + ' c' + str(event[2]) + ' p' + str(event[3]) + ' ')


            if event[0] == 'text_event':
              if karaoke:
                  this_channel_has_note = True

                  
                  tokens.append([event[2], event[1]])
                  
                  file.write('T' + ' d' + str(event[1]) + ' t' + str(event[2]) + ' ')


            if event[0] == 'copyright_text_event':
              if karaoke:
                  this_channel_has_note = True

                  
                  tokens.append([event[2], event[1]])
                  
                  file.write('R' + ' d' + str(event[1]) + ' t' + str(event[2]) + ' ')


            if event[0] == 'track_name':
                if karaoke:
                  this_channel_has_note = True

                  
                  tokens.append([event[2], event[1]])
                  
                  file.write('H' + ' d' + str(event[1]) + ' t' + str(event[2]) + ' ')


            if event[0] == 'instrument_name':
                if karaoke:
                  this_channel_has_note = True

                  
                  tokens.append([event[3], event[4], event[1]])
                  
                  file.write('I' + ' d' + str(event[1]) + ' t' + str(event[2]) + ' ')


            if event[0] == 'lyric':
                if karaoke:
                  this_channel_has_note = True

                  
                  tokens.append([event[2], event[1]])
                  file.write('L' + ' d' + str(event[1]) + ' t' + str(event[2]) + ' ')


            if event[0] == 'marker':
                if karaoke:
                  this_channel_has_note = True

                  
                  tokens.append([event[2], event[1]])
                  
                  file.write('M' + ' d' + str(event[1]) + ' t' + str(event[2]) + ' ')

            if event[0] == 'cue_point':
                if karaoke:
                  this_channel_has_note = True

                  
                  tokens.append([event[3], event[4], event[1]])
                  
                  file.write('U' + ' d' + str(event[1]) + ' t' + str(event[2]) + ' ')

            if event[0] == 'text_event_08':
                if karaoke:
                  this_channel_has_note = True

                  
                  tokens.append([event[2], event[1]])

                  file.write('+' + ' d' + str(event[1]) + ' t' + str(event[2]) + ' ')

            if event[0] == 'text_event_09':
                if karaoke:
                  this_channel_has_note = True

                  
                  tokens.append([event[2], event[1]])

                  file.write('&' + ' d' + str(event[1]) + ' t' + str(event[2]) + ' ')

            if event[0] == 'text_event_0a':
                if karaoke:
                  this_channel_has_note = True

                  
                  tokens.append([event[2], event[1]])                  
                  file.write('@' + ' d' + str(event[1]) + ' t' + str(event[2]) + ' ')

            if event[0] == 'text_event_0b':
                if karaoke:
                  this_channel_has_note = True

                  
                  tokens.append([event[2], event[1]])                  
                  file.write('#' + ' d' + str(event[1]) + ' t' + str(event[2]) + ' ')


            if event[0] == 'text_event_0c':
                if karaoke:
                  this_channel_has_note = True

                  
                  tokens.append([event[2], event[1]])                  
                  file.write('$' + ' d' + str(event[1]) + ' t' + str(event[2]) + ' ')

            if event[0] == 'text_event_0d':
                if karaoke:
                  this_channel_has_note = True

                  
                  tokens.append([event[2], event[1]])                  
                  file.write('%' + ' d' + str(event[1]) + ' t' + str(event[2]) + ' ')

            if event[0] == 'text_event_0e':
               if karaoke:
                  this_channel_has_note = True

                  
                  tokens.append([event[2], event[1]])                  
                  file.write('*' + ' d' + str(event[1]) + ' t' + str(event[2]) + ' ')

            if event[0] == 'text_event_0f':
              if karaoke:
                  this_channel_has_note = True

                  
                  tokens.append([event[2], event[1]])                  
                  file.write('=' + ' d' + str(event[1]) + ' t' + str(event[2]) + ' ')

            if event[0] == 'end_track':
                this_channel_has_note = True

                
                tokens.append([event[1]])                
                file.write('E' + ' d' + str(event[1]) + ' ')

            if event[0] == 'set_tempo':
              if advanced_events:
                if allow_tempo_changes:
                   this_channel_has_note = True
                
                   tokens.append([ event[2], event[1]])
                  
                   file.write('S' + ' d' + str(event[1]) + ' o' + str(event[2]) + ' ')

            if event[0] == 'smpte_offset':
              if advanced_events:
                this_channel_has_note = True
                
                tokens.append([event[3], event[4], event[1]])
                
                file.write('Y' + ' d' + str(event[1]) + ' g' + str(event[2]) + ' n' + str(event[3]) + ' s' + str(event[4]) + ' f' + str(event[5]) + ' e' + str(event[6]) +' ')

            if event[0] == 'time_signature':
              if advanced_events:
                this_channel_has_note = True

                
                tokens.append([event[3], event[4], event[1]])
                
                file.write('B' + ' d' + str(event[1]) + ' u' + str(event[2]) + ' y' + str(event[3]) + ' i' + str(event[4]) + ' j' + str(event[5]) +' ')


            if event[0] == 'key_signature':
              if advanced_events:
                this_channel_has_note = True
                
                tokens.append([event[3], event[2], event[1]])
                
                file.write('A' + ' d' + str(event[1]) + ' b' + str(event[2]) + ' q' + str(event[3]) + ' ')


            if event[0] == 'sequincer_specific':
              if advanced_events:
                  this_channel_has_note = True
                  
                  tokens.append([event[2], event[1]])
                  
                  file.write('D' + ' d' + str(event[1]) + ' x' + str(event[2]) + ' ')


            if event[0] == 'raw_meta_event':
              if advanced_events:
                  this_channel_has_note = True  

                  tokens.append([ event[2], event[1]]) 

                  file.write('E' + ' d' + str(event[1]) + ' z' + str(event[2]) + ' x' + str(event[2]) + ' ')

            if event[0] == 'sysex_f0':
              if advanced_events:
                  this_channel_has_note = True   

                  tokens.append([ event[2], event[1]])  

                  file.write('G' + ' d' + str(event[1]) + ' x' + str(event[2]) + ' ')

            if event[0] == 'sysex_f7':
              if advanced_events:
                  this_channel_has_note = True  

                  tokens.append([ event[2], event[1]]) 

                  file.write('!' + ' d' + str(event[1]) + ' x' + str(event[2]) + ' ')
                
            if event[0] == 'song_position':
              if advanced_events:
                  this_channel_has_note = True

                  tokens.append([ event[2], event[1]])

                  file.write('J' + ' d' + str(event[1]) + ' a' + str(event[2]) + ' ')

            if event[0] == 'song_select':
              if advanced_events:
                  this_channel_has_note = True 

                  tokens.append([ event[2], event[1]])

                  file.write('O' + ' d' + str(event[1]) + ' m' + str(event[2]) + ' ')

            if event[0] == 'tune_request':
              if advanced_events:
                  this_channel_has_note = True

                  tokens.append([ event[2], event[1]])

                  file.write('X' + ' d' + str(event[1]) + ' ')



        itrack += 1
        if not this_channel_has_note:
          print('Uknown Event: ', event[0])

        if this_channel_has_note and len(notes) > sample_length_in_MIDI_events:
          if enable_sampling:
            break
       

    file.close()     
       

dataset_addr = "Dataset"
files = os.listdir(dataset_addr)
for file in tqdm.auto.tqdm(files):
    path = os.path.join(dataset_addr, file)
    write_notes(path)

# Example Model Training (Layered LSTM NNs)

***

Default settings seem to produce best results so they are recommended. Do not make a model too big as it is not necessary and good results can be achieved with smaler models as well.

It is recommended to train for 100 epochs/0.8-0.9 accuracy


In [ ]:
#@title Define Constants and Functions { run: "auto" }
number_of_training_batches = 128 #@param {type:"slider", min:0, max:1536, step:16}
attention_sequence_length = 512 #@param {type:"slider", min:0, max:512, step:16}
embedding_size = 512 #@param {type:"slider", min:0, max:1024, step:16}
LSTM_layers_size = 256 #@param {type:"slider", min:0, max:1024, step:16}
dropout_rate = 0.2 #@param {type:"number"}
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dropout, TimeDistributed, Dense, Activation, Embedding

BATCH_SIZE = number_of_training_batches
SEQ_LENGTH = attention_sequence_length

def get_data_unique_chars_dicts(data):
    set_of_unique_chars = set(data)
    list_of_unique_chars = sorted(list(set_of_unique_chars))
    char_to_index = {}
    for i in range(len(list_of_unique_chars)):
        char_to_index[list_of_unique_chars[i]] = i
    index_to_char = {i: ch for (ch, i) in char_to_index.items()}
    return char_to_index, index_to_char

def read_batches(all_chars, unique_chars):
    length = all_chars.shape[0]
    batch_chars = int(length / BATCH_SIZE)
    for start in range(0, batch_chars - SEQ_LENGTH, SEQ_LENGTH):
        X = np.zeros((BATCH_SIZE, SEQ_LENGTH))
        Y = np.zeros((BATCH_SIZE, SEQ_LENGTH, unique_chars))
        for batch_index in range(0, BATCH_SIZE):  
            for i in range(0, SEQ_LENGTH):
                X[batch_index, i] = all_chars[batch_index * batch_chars + start + i]
                Y[batch_index, i, all_chars[batch_index * batch_chars + start + i + 1]] = 1
        yield X, Y

def make_model(batch_size, seq_length, num_of_unique_chars):
    model = Sequential()
    
    model.add(Embedding(input_dim = num_of_unique_chars, output_dim = embedding_size, 
                        batch_input_shape = (batch_size, seq_length))) 
    
    model.add(LSTM(LSTM_layers_size, return_sequences = True, stateful = True))
    model.add(Dropout(dropout_rate))
    
    model.add(LSTM(LSTM_layers_size, return_sequences = True, stateful = True))
    model.add(Dropout(dropout_rate))
    
    model.add(LSTM(LSTM_layers_size, return_sequences = True, stateful = True))
    model.add(Dropout(dropout_rate))
    
    model.add(TimeDistributed(Dense(num_of_unique_chars)))
    model.add(Activation("softmax"))
    
    return model

def train(model, model_name, num_of_epochs = 100):
  epoch_report = {}
      
  for epoch in range(num_of_epochs):
      print("Epoch {}/{}".format(epoch+1, num_of_epochs))

      average_epoch_loss = 0
      average_epoch_acc = 0
      counter = 0

      for i, (x, y) in enumerate(read_batches(chars_of_data, num_of_unique_chars)):
          if (i + 1) % 5:
            counter += 1
            batch_loss, batch_accuracy = model.train_on_batch(x, y)
            average_epoch_loss += batch_loss
            average_epoch_acc += batch_accuracy
            print("Batch: {}, Loss: {}, Accuracy: {}".format(i+1, batch_loss, batch_accuracy))
          else:
            batch_loss, batch_accuracy = model.test_on_batch(x, y)
            print(f"TEST Loss: {batch_loss}, Accuracy: {batch_accuracy}")

      epoch_report[epoch] = (average_epoch_loss / (counter), average_epoch_acc / (counter))
      
      if (epoch + 1) % 2 == 0:
          model.save_weights(f"Weights_{model_name}.h5")
          print(f'Saved Weights at epoch {epoch+1} to file Weights_{model_name}.h5')

  for epoch_num in epoch_report.keys():
    epoch_loss, epoch_acc = epoch_report[epoch_num]
    print(f"{epoch_num}\t{epoch_loss}\t{epoch_acc}")

In [ ]:
#@title Train the Model
number_of_training_epochs = 300 #@param {type:"slider", min:1, max:300, step:1}
with open('Dataset.txt', 'r') as data_file:
    data = data_file.read()

char_to_index, index_to_char = get_data_unique_chars_dicts(data)
num_of_unique_chars = len(char_to_index)
chars_of_data = np.asarray([char_to_index[c] for c in data], dtype = np.int32)

model = make_model(BATCH_SIZE, SEQ_LENGTH, num_of_unique_chars)
model.compile(loss = "categorical_crossentropy", optimizer = "Adamax", 
              metrics = ["accuracy"])

train(model, "MIDI-TXT-MIDI", number_of_training_epochs)


In [ ]:
#@title Save/Re-Save the Model from memory if needed
model.save('Weights_MIDI-TXT-MIDI.h5')

# Sampling
***

In [ ]:
#@title Define Functions
import tqdm.auto

def make_model_for_sampling(num_of_unique_chars):
    model = Sequential()
    
    model.add(Embedding(input_dim = num_of_unique_chars, output_dim = embedding_size, 
                        batch_input_shape = (1, 1))) 
  
    model.add(LSTM(LSTM_layers_size, return_sequences = True, stateful = True))
    model.add(Dropout(dropout_rate))
    
    model.add(LSTM(LSTM_layers_size, return_sequences = True, stateful = True))
    model.add(Dropout(dropout_rate))
    
    model.add(LSTM(LSTM_layers_size, stateful = True)) 
    model.add(Dropout(dropout_rate))
    
    model.add((Dense(num_of_unique_chars)))
    model.add(Activation("softmax"))
    
    return model

def adjust_seq(seq):
  count = 0
  for i in seq:
      count += 1
      if i == "\n":
          break
  beginning_part_removed = seq[count:]

  count = 0
  for i in beginning_part_removed:
      count += 1
      if i == "\n" and beginning_part_removed[count] == "\n":
          break
  ending_part_removed = beginning_part_removed[:count]
  return ending_part_removed

def generate_sequence(model_weights_address, initial_index, 
                      seq_length, is_abc=False):
    sequence_index = [initial_index]

    model = make_model_for_sampling(num_of_unique_chars)
    model.load_weights(model_weights_address)
    
    for i in tqdm.auto.tqdm(range(seq_length)):
        batch = np.zeros((1, 1))
        batch[0, 0] = sequence_index[-1]
        predicted_probs = model.predict_on_batch(batch).ravel()
        sample = np.random.choice(range(num_of_unique_chars), size = 1, p = predicted_probs)
        
        sequence_index.append(sample[0])
    
    seq = ''.join(index_to_char[c] for c in sequence_index)
    
    return seq

In [ ]:
#@title Generate Output
number_of_tokens_to_generate = 16000 #@param {type:"slider", min:0, max:16200, step:128}
seq = generate_sequence("Weights_MIDI-TXT-MIDI.h5", 0, number_of_tokens_to_generate)
print(seq)
with open('output.txt', 'w') as gen_song_file:
  gen_song_file.write(seq)

***

# MIDI to TXT Decoder

In [ ]:
#@title Convert to MIDI from TXT
number_of_ticks_per_quarter = 449 #@param {type:"slider", min:1, max:1440, step:8}

import MIDI
import tqdm.auto
notes = []
velocities = []
timings = []
durations = []

with open('/content/output.txt', 'r') as file:
    notestring=file.read()

score_note = notestring.split(" ")

score = score_note

i=0

song_score = [number_of_ticks_per_quarter, [['track_name', 0, b'Composed by Artificial Intelligence Model']]]

for i in tqdm.auto.tqdm(range(len(score))):

        # if the event is a blank, space, "eos" or unknown, skip and go to next event
        if score[i] in ["", " ", "<eos>", "<unk>"]:
            continue

        # if the event starts with 'end' indicating an end of note
        elif score[i][:2]=="@@":

            continue

        # in this block, we are looking for notes   
        else:
            # Look ahead to see if an end<noteid> was generated
            # soon after.  


            note_string_len = len(score[i])
            for j in range(1,200):
                if i+j==len(score):
                    break


            if score[i] == 'F':
              try:
                song_score[-1].append(['note_off', 
                                      int(score[i+1][1:]), 
                                      int(score[i+2][1:]),
                                      int(score[i+3][1:]),
                                      int(score[i+4][1:])])
                
              except:
                print("Unknown note: " + score[i])

            if score[i] == 'N':
              try:
                
                song_score[-1].append(['note_on', 
                                      int(score[i+1][1:]), #Duration
                                      int(score[i+2][1:]), #Channel
                                      int(score[i+3][1:]), #Note
                                      int(score[i+4][1:])]) #Velocity
                          
              except:
                print("Unknown note: " + score[i])


            if score[i] == 'K':
              try:
                song_score[-1].append(['key_after_touch', 
                                      int(score[i+1][1:]), 
                                      int(score[i+2][1:]),
                                      int(score[i+3][1:]),
                                      int(score[i+4][1:])])
              except:
                print("Unknown note: " + score[i])

            if score[i] == 'C':
              try:
                song_score[-1].append(['control_change',
                                      int(score[i+1][1:]), 
                                      int(score[i+2][1:]),
                                      int(score[i+3][1:]), #Controller
                                      int(score[i+4][1:])]) #ControlValue
              except:
                print("Unknown note: " + score[i])




            if score[i] == 'P':
              try:
                song_score[-1].append(['patch_change', 
                                      int(score[i+1][1:]), 
                                      int(score[i+2][1:]),
                                      int(score[i+3][1:])
                                      ])
              except:
                print("Unknown note: " + score[i])


            if score[i] == 'Z':
              try:
                song_score[-1].append(['channel_after_touch', 
                                      int(score[i+1][1:]), 
                                      int(score[i+2][1:]),
                                      int(score[i+3][1:])
                                      ])
              except:
                print("Unknown note: " + score[i])

            if score[i] == 'W':
              try:
                song_score[-1].append(['pitch_wheel_change', 
                                      int(score[i+1][1:]), 
                                      int(score[i+2][1:]),
                                      int(score[i+3][1:])
                                      ])

              except:
                print("Unknown note: " + score[i])

            if score[i] == 'T':
              try:
                song_score[-1].append(['text_event', 
                                      int(score[i+1][1:]), 
                                      int(score[i+2][1:])])
              except:
                print("Unknown note: " + score[i])

            if score[i] == 'R':
              try:
                song_score[-1].append(['copyright_text_event', 
                                      int(score[i+1][1:]), 
                                      int(score[i+2][1:])])
              except:
                print("Unknown note: " + score[i])

            if score[i] == 'H':
              try:
                song_score[-1].append(['track_name', 
                                      int(score[i+1][1:]), 
                                      int(score[i+2][1:])])
              except:
                print("Unknown note: " + score[i])


            if score[i] == 'I':
              try:
                song_score[-1].append(['instrument_name', 
                                      int(score[i+1][1:]), 
                                      int(score[i+2][1:])])
              except:
                print("Unknown note: " + score[i])

            if score[i] == 'L':
              try:
                song_score[-1].append(['lyric', 
                                      int(score[i+1][1:]), 
                                      int(score[i+2][1:])])
              except:
                print("Unknown note: " + score[i])

            if score[i] == 'M':
              try:
                song_score[-1].append(['marker', 
                                      int(score[i+1][1:]), 
                                      int(score[i+2][1:])])
              except:
                print("Unknown note: " + score[i])

            if score[i] == 'U':
              try:
                song_score[-1].append(['cue_point', 
                                      int(score[i+1][1:]), 
                                      int(score[i+2][1:])])
              except:
                print("Unknown note: " + score[i])

            if score[i] == '+':
              try:
                song_score[-1].append(['text_event_08', 
                                      int(score[i+1][1:]), 
                                      int(score[i+2][1:])])
              except:
                print("Unknown note: " + score[i])

            if score[i] == '&':
              try:
                song_score[-1].append(['text_event_09', 
                                      int(score[i+1][1:]), 
                                      int(score[i+2][1:])])
              except:
                print("Unknown note: " + score[i])

            if score[i] == '@':
              try:
                song_score[-1].append(['text_event_0a', 
                                      int(score[i+1][1:]), 
                                      int(score[i+2][1:])])
              except:
                print("Unknown note: " + score[i])

            if score[i] == '#':
              try:
                song_score[-1].append(['text_event_0b', 
                                      int(score[i+1][1:]), 
                                      int(score[i+2][1:])])
              except:
                print("Unknown note: " + score[i])

            if score[i] == '$':
              try:
                song_score[-1].append(['text_event_0c', 
                                      int(score[i+1][1:]), 
                                      int(score[i+2][1:])])
              except:
                print("Unknown note: " + score[i])

            if score[i] == '%':
              try:
                song_score[-1].append(['text_event_0d', 
                                      int(score[i+1][1:]), 
                                      int(score[i+2][1:])])
              except:
                print("Unknown note: " + score[i])

            if score[i] == '*':
              try:
                song_score[-1].append(['text_event_0e', 
                                      int(score[i+1][1:]), 
                                      int(score[i+2][1:])])
              except:
                print("Unknown note: " + score[i])

            if score[i] == '=':
              try:
                song_score[-1].append(['text_event_0f', 
                                      int(score[i+1][1:]), 
                                      int(score[i+2][1:])])
              except:
                print("Unknown note: " + score[i])

            if score[i] == 'E':
              try:
                song_score[-1].append(['end_track', 
                                      int(score[i+1][1:])])
              except:
                print("Unknown note: " + score[i])

            if score[i] == 'S':
              try:
                song_score[-1].append(['set_tempo', 
                                      int(score[i+1][1:]), 
                                      int(score[i+2][1:])])
              except:
                print("Unknown note: " + score[i])

            if score[i] == 'Y':
              try:
                song_score[-1].append(['smpte_offset',
                                      int(score[i+1][1:]), 
                                      int(score[i+2][1:]),
                                      int(score[i+3][1:]),
                                      int(score[i+4][1:]),
                                      int(score[i+5][1:]),
                                      int(score[i+6][1:])])
              except:
                print("Unknown note: " + score[i])                

            if score[i] == 'B':
              try:
                song_score[-1].append(['time_signature', 
                                      int(score[i+1][1:]), 
                                      int(score[i+2][1:]),
                                      int(score[i+3][1:]),
                                      int(score[i+4][1:]),
                                      int(score[i+5][1:])])

              except:
                print("Unknown note: " + score[i])


            if score[i] == 'A':
              try:
                song_score[-1].append(['key_signature', 
                                      int(score[i+1][1:]), 
                                      int(score[i+2][1:]),
                                      int(score[i+3][1:])])

              except:
                print("Unknown note: " + score[i])



            if score[i] == 'D':
              try:
                song_score[-1].append(['sequencer_specific', 
                                      int(score[i+1][1:]), 
                                      int(score[i+2][1:])])

              except:
                print("Unknown note: " + score[i])

            if score[i] == 'E':
              try:
                song_score[-1].append(['raw_meta_event', 
                                      int(score[i+1][1:]), 
                                      int(score[i+2][1:])])

              except:
                print("Unknown note: " + score[i])

            if score[i] == 'G':
              try:
                song_score[-1].append(['sysex_f0', 
                                      int(score[i+1][1:]), 
                                      int(score[i+2][1:])])

              except:
                print("Unknown note: " + score[i])

            if score[i] == '!':
              try:
                song_score[-1].append(['sysex_f7', 
                                      int(score[i+1][1:]), 
                                      int(score[i+2][1:])])

              except:
                print("Unknown note: " + score[i])


            if score[i] == 'J':
              try:
                song_score[-1].append(['song_position', 
                                      int(score[i+1][1:]), 
                                      int(score[i+2][1:])])

              except:
                print("Unknown note: " + score[i])

            if score[i] == 'O':
              try:
                song_score[-1].append(['song_select', 
                                      int(score[i+1][1:]), 
                                      int(score[i+2][1:])])

              except:
                print("Unknown note: " + score[i])

            if score[i] == 'X':
              try:
                song_score[-1].append(['tune_request', 
                                      int(score[i+1][1:]), 
                                      int(score[i+2][1:])])

              except:
                print("Unknown note: " + score[i])

midi_data = MIDI.opus2midi(song_score)
with open('output.mid', 'wb') as midi_file:
    midi_file.write(midi_data)

MIDI.score2stats(song_score)